### XGBoost Endpoint - with Multiple Models hosted on same instance

In [3]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import json

import boto3
import re
from sagemaker import get_execution_role
import sagemaker

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [4]:
# We need to specify the location of each model - with relative path

targetModels = ["xgboost-bikerental-hyper-one-2024-05-08-23-26-31-842/output/model.tar.gz",
                "xgboost-bikerental-hyper-two-2024-05-08-23-29-29-045/output/model.tar.gz"
]

In [12]:
# Create a predictor and point to an existing endpoint
endpoint_name = 'xgboost-bikerental-hyper'
predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)
predictor.serializer = CSVSerializer()

In [6]:
#Read test data

df_all = pd.read_csv('bike_test_rev3.csv')

In [7]:
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4


In [8]:
arr_test = df_all[df_all.columns[1:]].values

In [10]:
arr_test

array([[ 1.,  0.,  1., ..., 20.,  3.,  0.],
       [ 1.,  0.,  1., ..., 20.,  3.,  1.],
       [ 1.,  0.,  1., ..., 20.,  3.,  2.],
       ...,
       [ 1.,  0.,  1., ..., 31.,  0., 21.],
       [ 1.,  0.,  1., ..., 31.,  0., 22.],
       [ 1.,  0.,  1., ..., 31.,  0., 23.]])

In [11]:
arr_test.shape

(6493, 13)

#### Invoke Model one

In [13]:
# target_model = path to the model artifcat

result = predictor.predict(arr_test[:5],
                          target_model= targetModels[0])
print(result)

b'[2.332122802734375, 1.900559663772583, 1.6316102743148804, 1.0376965999603271, 0.9574325680732727]'


#### Invoke Model two

In [15]:
# target_model = path to the model artifcat

result = predictor.predict(arr_test[:5],
                          target_model= targetModels[1])
print(result)

b'[2.489320993423462, 1.7995193004608154, 1.92416250705719, 1.335949420928955, 1.188660740852356]'


### Split the input data into chunks

There are thousands of rows in test dataset<br>
When communicating over internet, it is a good idea to split the data into chunks to prevent payload and timeout error

In [17]:
# splitting using regular expression as xgboost 1.7-2 will return
# predicted values with inconsistent delimiters (comma, newline, or both)

# patter looks for one or more non numeric characters

pattern = r'[^0-9.]+'

def inference_by_version(targetModel=None):
    
    predictions = []
    for arr in np.array_split(arr_test,10):
        result = predictor.predict(arr, target_model=targetModel)
        result = re.split(pattern, result.decode("utf-8"))
        print(arr.shape)
        predictions += [float(r) for r in result if r!=""]
        
    return predictions

### Use all available variants

In [18]:
print("model one inference")
df_all['count_hyper_one'] = np.expm1(inference_by_version(targetModel=targetModels[0]))

print("model two inference")
df_all['count_hyper_two'] = np.expm1(inference_by_version(targetModel=targetModels[1]))


model one inference
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
model two inference
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)


In [20]:
df_all[['count_hyper_one','count_hyper_two']].describe()

,count_hyper_one,count_hyper_two
count,6493.000000,6493.000000
mean,183.412356,184.068837
std,171.869899,175.669847
min,0.419208,0.718338
25%,41.106623,39.230815
50%,141.825509,138.852125
75%,273.490279,272.472951
max,942.929876,902.084974


In [21]:
df_all[['count_hyper_one','count_hyper_two']].head()

,count_hyper_one,count_hyper_two
0,9.299783,11.053089
1,5.689637,5.046740
2,4.112100,5.849410
3,1.822708,2.803605
4,1.605000,2.282682


In [22]:
# delete Endpoint
predictor.delete_endpoint()